# Image Data Augmentation Using Deep Convolutional Generative Adversarial Network (DCGAN)
# Model - Generate Images

## Import Libraries

In [1]:
import numpy as np
import keras
from keras import layers
from PIL import Image

Using TensorFlow backend.


## Load the selected trained GAN using the weights saved to file during model training

In [2]:
# Generator
latent_dim = 128
height = 128
width = 128
channels = 5

generator_input = keras.Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = layers.Dense(256 * 64 * 64)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((64, 64, 256))(x)

# Then, add a convolution layer
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsample to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# Few more conv layers
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produce a 32x32 1-channel feature map
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1048576)           135266304 
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1048576)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 256)       1638656   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 128, 128, 256)     1048832   
__________

In [3]:
# Discriminator
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

# One dropout layer - important trick!
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 5)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 126, 126, 128)     5888      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 126, 126, 128)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 62, 62, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 128)       262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 30, 30, 128)       0         
__________

In [4]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0002, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [5]:
# Load the weights from file.
gan.load_weights('.\data\\gan_model\\gan4500.h5')

## Generate the GAN images and mask. Save to file.

In [6]:
import string
from random import *

max_char = 12
allchar = string.ascii_letters + string.digits
num_generated_images = 100
bw_threshold = 230
save_dir = '.\data\\gan_generated_images\\'

# Sample random points in the latent space
random_latent_vectors = np.random.normal(size=(num_generated_images, latent_dim))

# Decode them to fake images
generated_images = generator.predict(random_latent_vectors)
    
for i in range(generated_images.shape[0]):

    # Separate the image and mask
    img_arr = (generated_images[i][:, :, :4] * 255.).astype(np.uint8)
    mask_arr = (generated_images[i][:, :, 4] * 255.).astype(np.uint8)
    mask_arr[mask_arr > bw_threshold] = 255
    mask_arr[mask_arr < 255] = 0
        
    # Save as png files.
    img = Image.fromarray(img_arr, mode='RGBA')
    a_filename = "".join(choice(allchar) for x in range(max_char))
    img.save(save_dir + a_filename + '_image.png')
    mask = Image.fromarray(mask_arr, mode='L')
    mask.save(save_dir + a_filename + '_mask.png')